<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/Video/VideoColorizer_with_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<font color='blue'> Video Colorizer </font>**

#◢ DeOldify - Colorize your own videos!


_FYI: This notebook is intended as a tool to colorize gifs and short videos, if you are trying to convert longer video you may hit the limit on processing space. Running the Jupyter notebook on your own machine is recommended (and faster) for larger video sizes._

####**Credits:**

Big special thanks to:

Robert Bell for all his work on the video Colab notebook, and paving the way to video in DeOldify!

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


#◢ Git clone and install DeOldify

In [ ]:
###################################################### 
import os 
os.system('curl https://raw.githubusercontent.com/bxck75/colab_starter/main/setup.py -o /content/setup.py') 
 
######################################################

0

In [ ]:
os.system('python /content/setup.py')

0

##Get videos

In [ ]:
import requests,pprint
from bs4 import BeautifulSoup
import requests, os, sys


def GetCollections(sources_list):
    video_collections = []
    for collection_name,url in sources_list:
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        myas = soup.find_all("a", {"class": "stealth download-pill"}, href=True)
        video_urls = []
        for a in myas:
            #print("Found the URL:", a['href'])
            split1 = a['href'].split('/')
            filename = split1.pop()
            split2 = filename.split('%20-%20')
            title = split2.pop()
            filename = filename.replace('%20','_')
            title = title.replace('%20','_')

            video_urls.append(['https://archive.org'+a['href'], filename, title])

        video_collections.append([collection_name, video_urls])

    return video_collections

def download_videos(video_list, output_dir='/content/output'):
    # make output path
    os.makedirs(output_dir, exist_ok=True)
    # for every collection
    for collection_name, video_detail_list in video_list:
        collection_path = os.path.join(output_dir,collection_name)
        # make collection path
        os.makedirs(collection_path, exist_ok=True)
        pprint.pprint("downloading "+collection_name)
        # for every video dowload and save
        for video_url,video_filename,video_title in video_detail_list:      
            r = requests.get(video_url)
            with open(os.path.join(collection_path,video_filename), 'wb') as fd:
                fd.write(r.content)


sources = [
#            ['japan_latenight','https://archive.org/details/CNJapanLateNightBW200305'],
            ["popeye","https://archive.org/details/popeye-collection"],
#            ["bettyboop","https://archive.org/details/BettyBoopCartoonsCartoonClassicsVol.1"]
           ]

download_videos(sorted(GetCollections(sources)))
#pprint.pprint(video_download_list)


In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2603, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 2603 (delta 83), reused 209 (delta 71), pack-reused 2346
Receiving objects: 100% (2603/2603), 69.71 MiB | 21.88 MiB/s, done.
Resolving deltas: 100% (1164/1164), done.


In [ ]:
os.chdir('/content/GitGo/DeOldify')

#◢ Setup

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

In [ ]:
!pip install -r requirements-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654003 sha256=b8521b37911fa5b23378c0fd3f732e83fc8c5755622b364f7408c3573399c8d0
  Stored in directory: /root/.cache/pip/wheels/cb/c7/a6/2d7a113c4885dc0f4eacd8f41095763181c0b9a18223ac7533
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.4.0
    Uninstalling imgaug-0.4.0:
      Successfully uninstalled imgaug-0.4.0


In [ ]:
!pip install pillow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 6.2.2
    Uninstalling Pillow-6.2.2:
      Successfully uninstalled Pillow-6.2.2


In [ ]:
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

ImportError: ignored

In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2023-06-02 03:28:57--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.199.36.8, 2400:52e0:1e00::865:1
Connecting to data.deepai.org (data.deepai.org)|138.199.36.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘./models/ColorizeVideo_gen.pth’

./models/ColorizeVi 100%[===================>] 833.57M  4.68MB/s    in 2m 57s  

2023-06-02 03:31:55 (4.70 MB/s) - ‘./models/ColorizeVideo_gen.pth’ saved [874066230/874066230]



In [ ]:
colorizer = get_video_colorizer()

NameError: ignored

#◢ Instructions

### source_url
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.  Full list here: https://ytdl-org.github.io/youtube-dl/supportedsites.html NOTE: If you want to use your own video, upload it first to a site like YouTube. 

### render_factor
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out.

### watermarked
Selected by default, this places a watermark icon of a palette at the bottom left corner of the image.  This is intended to be a standard way to convey to others viewing the image that it is colorized by AI. We want to help promote this as a standard, especially as the technology continues to improve and the distinction between real and fake becomes harder to discern. This palette watermark practice was initiated and lead by the company MyHeritage in the MyHeritage In Color feature (which uses a newer version of DeOldify than what you're using here).

### How to Download a Copy
Simply right click on the displayed video and click "Save video as..."!

## Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.
2. If videos are taking way too much time for your liking, running the Jupyter notebook VideoColorizer.ipynb on your own machine (with DeOldify installed) will generally be much faster (as long as you have the hardware for it).   
3. Longer videos (running multiple minutes) are going to have a rough time on Colabs. You'll be much better off using a local install of DeOldify instead in this case.

## Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Files
2. If you don't see the 'DeOldify' folder, click "Refresh"
3. By default, rendered video will be in /DeOldify/video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.

#◢ Colorize!!

In [ ]:
source_url = 'https://ia801009.us.archive.org/24/items/BimbosInitiation_201802/Betty%20Boop%20Bimbo%27s%20Initiation%201932%20HD%20Grim%20Natwick%20Fleischer%20Studios.mp4' #@param {type:"string"}
render_factor = 23  #@param {type: "slider", min: 5, max: 40}
watermarked = True #@param {type:"boolean"}

if source_url is not None and source_url !='':
    video_path = colorizer.colorize_from_url(source_url, 'video.mp4', render_factor, watermarked=watermarked)
    show_video_in_notebook(video_path)
else:
    print('Provide a video url and try again.')

## See how well render_factor values perform on a frame here

In [ ]:
for i in range(10,40,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

---
#⚙ Recommended video and gif sources 
* [/r/Nickelodeons/](https://www.reddit.com/r/Nickelodeons/)
* [r/silentmoviegifs](https://www.reddit.com/r/silentmoviegifs/)
* https://twitter.com/silentmoviegifs 